<h3>Tests for IPknot</h3>

In [1]:
import pandas as pd

import time
from selenium import webdriver
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup

In [2]:
aptabase = pd.read_csv('Aptamers_2D.csv')
aptabase.head()

,PDB_ID,Sequence,2D,Len,Structure,RNAfold_ref
0,1JVE,cctaattataacgaagttataattagg,(((((((((((((.))))))))))))),27,Triplex-DNA,1.0
1,1NGU,ctctccttgtatttcttacaaaaagag,(((((((((((.....))))))))))),27,Hairpin with pseudoknots,1.0
2,2OEY,ccatcgtctacctttggtaggatgg,(((((..(((((...)))))))))),25,Hairpin with loops,1.0
3,2VWJ,aauggagacacggcttttgccgtgtc,......((((((((....)))))))),26,Hairpin with dangling ends,1.0
4,3THW,cctctatctgaagccgatcgatgaagcatcgatcgcacagcttcag...,((((((((((((((((((((((((.))))))))))....)))))))...,53,Hairpin with loops,1.0


In [3]:
max(list(aptabase['Len']))

57

For sequences <80 nt it is recommended to use NUPACk engine, although it has more computational cost and disallow refinement for pseudoknotted structures



In [4]:
fullref = ''.join(aptabase['2D'])
fullref.count('{')

19

if structure can contain '{' and '}' it is recommmended to use level 3 of prediction

In [5]:
'aaaattttvv'.replace('t', 'u')

'aaaauuuuvv'

In [17]:
def ipknoter_v1(seq):

    driver = webdriver.Chrome()


    s = seq.replace('t', 'u')

    # Open web-server
    driver.get('http://ws.sato-lab.org/rtips/ipknot++/')  # Replace with your target URL
    
    # type sequence
    input_seq = driver.find_element(By.NAME, "seq")
    input_seq.send_keys(s)

    #settings
    # select_level = driver.find_element(By.NAME, 'level')
    # select_level.selectByValue('3')

    # select_engine = driver.find_element(By.NAME, 'model')
    # select_engine.select_by_value('NUPACK')

    select_level3 = driver.find_element(By.XPATH, '/html/body/div/form/ul/li[1]/select/option[3]')
    select_level3.click()

    select_nupack = driver.find_element(By.XPATH, '/html/body/div/form/ul/li[2]/select/option[6]')
    select_nupack.click()

    #click pred button
    predbutton = driver.find_element(By.XPATH, "/html/body/div/form/p[3]/input[1]")
    predbutton.click()

    # Wait for the page to load 
    time.sleep(10)

    
    # Get the resulting HTML
    result_html = driver.page_source
    driver.quit()

   
    
    #we need text from xpath /html/body/div/div[2]/pre[3]
    # Создаём объект BeautifulSoup
    soup = BeautifulSoup(result_html, 'html.parser')

    # CSS-селектор, соответствующий вашему XPath
    css_selector = 'body > div > div:nth-of-type(2) > pre:nth-of-type(3)'

    # Находим элемент
    element = soup.select_one(css_selector)
    text = element.get_text()
    return text.split()[1]

In [18]:
ipknoter_v1('aaaagggtttt')

'.((.....)).'

In [21]:
#ipknotv1 - level3, nupack model
seqs = list(aptabase['Sequence'])
ipknotv1_preds = []
for seq in seqs:
    ipknotv1_preds.append(ipknoter_v1(seq))
    print(len(ipknotv1_preds), '-', end=' ')

1 - 2 - 3 - 4 - 5 - 6 - 7 - 8 - 9 - 10 - 11 - 12 - 13 - 14 - 15 - 16 - 17 - 18 - 19 - 20 - 21 - 22 - 23 - 24 - 25 - 26 - 27 - 28 - 29 - 30 - 31 - 32 - 33 - 34 - 35 - 36 - 37 - 38 - 39 - 40 - 41 - 42 - 43 - 44 - 45 - 46 - 47 - 48 - 49 - 50 - 51 - 52 - 53 - 54 - 55 - 56 - 57 - 58 - 59 - 60 - 61 - 62 - 63 - 64 - 65 - 66 - 67 - 68 - 69 - 

In [23]:
def accuracy_2D(pred, ref):

    count = 0 
    for k in range(len(pred)):

        if pred[k] == ref[k]:
            count += 1

        elif (pred[k]=='+') and (ref[k] in ('(', ')', '[', ']', '{', '}')):
            count += 1

        # elif (pred[k] in ('[', '{')) and (ref[k] in ('[', '{')):
        #     count +=1

        # elif (pred[k] in (']', '}')) and (ref[k] in (']', '}')):
        #     count +=1

    return round(count/len(pred), 2)

In [24]:
ipknotv1_accuracy_points = []
ref2d = list(aptabase['2D'])

for i in range(len(ref2d)):
    ipknotv1_accuracy_points.append(accuracy_2D(ipknotv1_preds[i], ref2d[i]))

In [28]:
#% of accurate structures
acc_counter = 0
for ac in ipknotv1_accuracy_points:
    if ac >= 0.85:
        acc_counter +=1

print('% of Accurate structures (accuracy >= 0.85) =', 100*(acc_counter/len(ipknotv1_accuracy_points)))

#Full accuracy
nt_accuracy = accuracy_2D(''.join(ipknotv1_preds), ''.join(ref2d))
print('Nucleotide Accuracy=', nt_accuracy)

% of Accurate structures (accuracy >= 0.85) = 33.33333333333333
Nucleotide Accuracy= 0.66


In [29]:
aptabase['IPknotv1_2D'] = pd.Series(ipknotv1_preds)
aptabase['IPknotv1'] = pd.Series(ipknotv1_accuracy_points)
aptabase

,PDB_ID,Sequence,2D,Len,Structure,RNAfold_ref,IPknotv1_2D,IPknotv1
0,1JVE,cctaattataacgaagttataattagg,(((((((((((((.))))))))))))),27,Triplex-DNA,1.00,((((((((((((...)))))))))))),0.93
1,1NGU,ctctccttgtatttcttacaaaaagag,(((((((((((.....))))))))))),27,Hairpin with pseudoknots,1.00,((((...((((.....))))...)))),0.78
2,2OEY,ccatcgtctacctttggtaggatgg,(((((..(((((...)))))))))),25,Hairpin with loops,1.00,(((((..(((((...)))))))))),1.00
3,2VWJ,aauggagacacggcttttgccgtgtc,......((((((((....)))))))),26,Hairpin with dangling ends,1.00,......((((((((....)))))))),1.00
4,3THW,cctctatctgaagccgatcgatgaagcatcgatcgcacagcttcag...,((((((((((((((((((((((((.))))))))))....)))))))...,53,Hairpin with loops,1.00,(((((((((((((((((((((((...)))))))))....)))))))...,0.96
...,...,...,...,...,...,...,...,...
64,4U5M,tggtggtggtggttgtggtggtggtgtt,(([.)])([.)]..(.[).](.[).]..,28,G-quadruplex,0.71,............................,0.36
65,2N3M,tggtggtggttgttgtggtggtggtggt,(([.)].[{.)]..}.([.)].([.)].,28,G-quadruplex,0.71,............................,0.36
66,201D,ggggttttggggttttggggttttgggg,(((((.)[))))....((((...])))),28,G-quadruplex,1.00,............................,0.29
67,5J6U,ggggtttggggttttggggaagggg,((((.[.))))....(((]...))),25,G-quadruplex,1.00,...........((.....)).....,0.24


In [30]:
def TN(pred, ref):

    counter = 0
    for i in range(len(ref)):
        if (pred[i] == '.') and (ref[i]=='.'):
            counter +=1

    return counter

def FN(pred, ref):

    counter = 0
    for i in range(len(ref)):
        if (pred[i] == '.') and (ref[i]!='.'):
            counter +=1

    return counter


   
def find_basepairs(s):
    stack = []
    #result = {}
    result = []

    for index, char in enumerate(s):
        if char == '(':
            # Запоминаем индекс открывающей скобки
            stack.append(index)
        elif char == ')':
            # Если нашли закрывающую скобку, сопоставляем с последней открывающей
            if stack:
                open_index = stack.pop()
                #result[open_index] = index
                result.append((open_index, index))

    return result

def find_knotpairs(s):
    stack = []
    #result = {}
    result = []

    for index, char in enumerate(s):
        if char == '[':
            # Запоминаем индекс открывающей скобки
            stack.append(index)
        elif char == ']':
            # Если нашли закрывающую скобку, сопоставляем с последней открывающей
            if stack:
                open_index = stack.pop()
                #result[open_index] = index
                result.append((open_index, index))

    return result

def find_otherpairs(s):
    stack = []
    #result = {}
    result = []

    for index, char in enumerate(s):
        if char == '{':
            # Запоминаем индекс открывающей скобки
            stack.append(index)
        elif char == '}':
            # Если нашли закрывающую скобку, сопоставляем с последней открывающей
            if stack:
                open_index = stack.pop()
                #result[open_index] = index
                result.append((open_index, index))

    return result

def TP(pred,ref):

    counter=0

    pred_basepairs = find_basepairs(pred)
    pred_knotpairs = find_knotpairs(pred)
    pred_otherpairs = find_otherpairs(pred)

    ref_basepairs = find_basepairs(ref)
    ref_knotpairs = find_knotpairs(ref)
    ref_otherpairs = find_otherpairs(ref)

    for b in pred_basepairs:
        if b in ref_basepairs:
            counter += 2

    for k in pred_knotpairs:
        if k in ref_knotpairs:
            counter += 2

    for o in pred_otherpairs:
        if k in ref_otherpairs:
            counter += 2

    return counter

    



In [31]:
tp = TP(''.join(ipknotv1_preds), ''.join(ref2d))
fn = FN(''.join(ipknotv1_preds), ''.join(ref2d))
tn = TN(''.join(ipknotv1_preds), ''.join(ref2d))
fp = len(''.join(ipknotv1_preds)) - (tp+fn+tn)

Precision = tp/(tp+fp)
Recall = tp/(tp+fn)
F1 = 2*Precision*Recall/(Precision+Recall)

print('Precision =', round(Precision, 2))
print('Recall =', round(Recall, 2))
print('F1-score =', round(F1, 2))

Precision = 0.76
Recall = 0.6
F1-score = 0.67


In [32]:
set('[]{}')

{'[', ']', '{', '}'}

In [42]:
def pseudoknots_accuracy(pred, ref):

    count = 0
    pseudo = ('[', ']', '{','}')
    pseudo_n = 0
    
    for k in range(len(ref)):
        if (ref[k] in pseudo):
            pseudo_n += 1
            if ref[k] == pred[k]:
                count += 1

    if pseudo_n != 0:
        return round(count/pseudo_n, 2)
    
    else:
        return -1

pseudoknots_accuracy('((((.[].))))', '(((([[]])))))')


0.5

In [40]:
print('Pseudoknots accuracy =', pseudoknots_accuracy(''.join(ipknotv1_preds), ''.join(ref2d)))

Pseudoknots accuracy = 0.0


In [46]:
for i in range(len(ipknotv1_preds)):
    p = pseudoknots_accuracy(ipknotv1_preds[i], ref2d[i])
    if p != -1:
        print(ipknotv1_preds[i], ref2d[i], sep='\n')
        print('----------------')

((((......((..((([[....(((....)))......)))]]))......))))..
(((((()([(((...))))..{)))))]}(((((()([(((...))))..{)))))]}
----------------
((((((..((..[[.))..]]...))))))
(((((((([[).((]....))].)))))))
----------------
.(((.....))).(((((.....)))))....
((((....([[.)(((((]](.))))))))))
----------------
(((....)))(((((((((((((....)))))))))))))
(((((((((([[[[[[[[[[))))))))))]]]]]]]]]]
----------------
........(((((.....))))).......
(([..))((((((((.))))))))((.]))
----------------
((((((.................)))))).
(((((((((..)).[)](..)..)))))).
----------------
.(((((.....)))))................
((((((((.))))))))([)(].[)(...].)
----------------
...(((((((((.........)))))))))....
(([{.)]}.[.)]([(((((((...))))))))]
----------------
.........(((((.....)))))........
()([{.)]}.(((((...))))).([{.)]}.
----------------
((((......))))..(((.....)))
(((([(](.[)...).(.])..).)))
----------------
......((...............)).
.[{([{...)]}.(.(((.]})))).
----------------
..((...((....))...))......
...[([)(.](](.)[)(...]

In all cases IPknot v1 (level=3, model=NUPACK) failed to predict pseudokots, although prediction of classical secondary structure elements were compatible with RNAfold results. 

There are many of non-canonical base pairs in pseudoknots, thats why I consider that choosing another settings will not improve IPknot performance


In [50]:
aptabase.to_csv('IPknot_benchmarking.csv')